In [ ]:
import os
pgurl = os.environ["REMOTE_POSTGRES_URL"]
from sqlalchemy import create_engine, text

results = None
with create_engine(pgurl).connect() as conn:
    result = conn.execute(text("select * from users where id=15972102"))
    results = result.all()
results

In [ ]:
import numpy as np
USER_DTYPE = np.dtype([
    ("uid", np.uint32), 
    ("username", object), 
    ("firstname", object), 
    ("lastname", str), 
    ("profile", str), 
    ("access_token", str), 
    ("measurement_preference", str),
    ("city", str),
    ("state", str),
    ("country", str),
    ("email", str),
    ("dt_last_active", str),
    ("dt_indexed", str),
    ("app_activity_count", str),
    ("share_profile", bool),
    ("extra", str)
])
results2 = np.zeros(len(results), dtype=USER_DTYPE)

In [ ]:
import motor.motor_asyncio
mongo_url = os.environ.get("REMOTE_MONGODB_URL")
client = motor.motor_asyncio.AsyncIOMotorClient(mongo_url)

In [ ]:
db = client.get_default_database()
db

In [ ]:
import asyncio
from pprint import pprint
names = await db.list_collection_names()
pprint(names)

cursor = db["index"].find()
index = await cursor.to_list(length=100)

In [ ]:
index

In [ ]:
activities = await db["activities"].find().limit(10).to_list(length=10)

In [ ]:
import msgpack as mp
import numpy as np 
import polyline

def rll_decode(rll_encoded, first_value=0):
    running_sum = first_value
    out_list = [first_value]

    for el in rll_encoded:
        if isinstance(el, list) and len(el) == 2:
            val, num_repeats = el
            for i in range(num_repeats):
#                 running_sum += val
#                 out_list.append(running_sum)
                out_list.append(val)
        else:
            out_list.append(el)
#             running_sum += el
#             out_list.append(running_sum)

    return out_list

LATLNG_DTYPE = np.dtype([("dt", np.uint8), ("lat", np.float32), ("lng", np.float32)])
A = mp.unpackb(activities[0]["mpk"])

lat, lng = zip(*polyline.decode(A["polyline"]))
dt = rll_decode(A["time"])
tlatlng = np.array(list(zip(dt, lat, lng)), LATLNG_DTYPE)
                

In [ ]:
tlatlng, tlatlng.nbytes

In [ ]:
activities[0]["mpk"], len(activities[0]["mpk"])

In [ ]:
import aioredis
redis_url = os.environ.get("REMOTE_REDIS_URL")
redis = aioredis.from_url(redis_url)

In [ ]:
key = await redis.randomkey()
val = await redis.get(key)

In [ ]:
key, mp.unpackb(val)